<a href="https://colab.research.google.com/github/dmitriy-kuleshov/feedback-analyser-project/blob/main/analyser_for_service.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Удаление неподходящих отзывов**

In [12]:
import json

# Функция для проверки "пустых" и коротких отзывов
def is_informative(review):
    # Проверяем количество заполненных полей
    filled_fields = sum(bool(review[key]) for key in ["text", "pros", "cons"])

    # Считаем суммарную длину всех полей
    total_length = len(review["text"].strip()) + len(review["pros"].strip()) + len(review["cons"].strip())

    # Отбрасываем, если заполнено ≤ 1 поля или суммарная длина < 50 символов
    return filled_fields > 1 and total_length >= 50

# Читаем JSON
with open("10.json", "r", encoding="utf-8") as file:
    data = json.load(file)

# Фильтруем отзывы
filtered_data = [review for review in data if is_informative(review)]

# Записываем обратно
with open("filtered_data.json", "w", encoding="utf-8") as file:
    json.dump(filtered_data, file, indent=4, ensure_ascii=False)

print("Фильтрация завершена! Результат сохранён в filtered_data.json")


Фильтрация завершена! Результат сохранён в filtered_data.json


**1. Модель обучения**

---



In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import json
import numpy as np
import re

MODEL_NAME = "blanchefort/rubert-base-cased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
model.eval()

def preprocess_text(review):
    parts = []
    if review.get("productValuation"):
        parts.append(f"Оценка: {review['productValuation']} из 5.")
    if review["text"]:
        parts.append(review["text"])
    if review["pros"]:
        parts.append(f"Плюсы: {review['pros']}")
    if review["cons"]:
        parts.append(f"Минусы: {review['cons']}")
    return " ".join(parts).strip()

def rule_based_boost(text, sentiment, scores):
    text_lower = text.lower()

    negative_keywords = [
    r"не\s+рекомендую", r"не\s+советую", r"полная.+шляпа",
    r"все.+вранье", r"сломался", r"не\s+работает", r"развалился", r"обман",
    r"покупайте.+скупой.+дважды", r"говно", r"очень плохо", r"очень плохое",
    r"отвратительно", r"отвратительное", r"отвратительный", r"отвратительная",
    r"рваный", r"ужас", r"жесть", r"кошмар", r"разочарование", r"не стоящий",
    r"ужасный", r"убогий", r"не работает", r"не включается", r"не отвечает",
    r"не работает корректно", r"неисправный", r"часто зависает", r"перегревается",
    r"плохо сделано", r"не соответствует описанию", r"не оправдал ожиданий",
    r"плохое качество", r"низкое качество", r"хлипкий", r"бракованный",
    r"не стоит своих денег", r"не соответствует рекламе", r"обман", r"надувательство",
    r"развод", r"не покупайте", r"ужасный опыт"
    ]


    positive_keywords = [
    r"всё отлично", r"работает хорошо", r"супер", r"советую", r"рекомендую",
    r"прекрасно", r"очень доволен", r"идеально", r"качество.+отличное", r"понравился",
    r"нравится", r"классно", r"классный", r"классное", r"топ", r"топово", r"топовый",
    r"топовое", r"топовая", r"классная", r"суперски", r"суперский", r"суперская",
    r"суперское", r"идеальный", r"отличный", r"в восторге", r"замечательно", r"потрясающе",
    r"восхищён", r"удивительно", r"шикарно", r"бомбически", r"огонь", r"невероятно",
    r"отлично", r"цена/качество на высоте", r"не пожалел", r"без нареканий", r"превосходно",
    r"удобный", r"качественный", r"легкий в использовании", r"практичный", r"хороший"
    ]


    # Если есть негативные слова — повышаем уверенность в негативе
    if any(re.search(pat, text_lower) for pat in negative_keywords):
        return "негатив"

    # Если есть позитивные слова — усиливаем позитив
    if any(re.search(pat, text_lower) for pat in positive_keywords):
        return "позитив"

    return sentiment

def analyze_sentiment(text):
    tokens = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        output = model(**tokens)

    scores = torch.nn.functional.softmax(output.logits, dim=-1).numpy()[0]
    num_labels = len(scores)

    # Подбираем нужные ярлыки
    if num_labels == 2:
        labels = ["негатив", "позитив"]
    else:
        labels = ["негатив", "нейтральный", "позитив"]

    # Основная логика определения
    confidence = np.max(scores)

    # По умолчанию — определяем по уверенности
    if confidence < 0.5:
        sentiment = "нейтральный"
    else:
        sentiment = labels[np.argmax(scores)]

    # При 3 классах — уточняем с помощью разницы между позитивом и негативом
    if num_labels == 3:
      delta = scores[2] - scores[0]
      if delta > 0.15:
        sentiment = "позитив"
      elif delta < -0.15:
        sentiment = "негатив"
      else:
        sentiment = "нейтральный"


    return sentiment, scores


def adjust_sentiment_based_on_rating(sentiment, rating):
    if rating in [4, 5]:
        if sentiment == "негатив":
            sentiment = "нейтральный"
        elif sentiment == "нейтральный":
            sentiment = "позитив"
        else:
            sentiment = "позитив"

    elif rating == 3:
        sentiment = "нейтральный"

    elif rating in [1, 2]:
        if sentiment == "позитив":
            sentiment = "нейтральный"
        elif sentiment == "нейтральный":
            sentiment = "негатив"
        else:
            sentiment = "негатив"

    return sentiment

# Читаем JSON с отзывами
with open("filtered_data.json", "r", encoding="utf-8") as file:
    reviews = json.load(file)

results = []
for review in reviews:
    full_text = preprocess_text(review)
    rating = review.get("productValuation", 3)

    if full_text:
        sentiment, scores = analyze_sentiment(full_text)
        original_sentiment = sentiment
        sentiment = rule_based_boost(full_text, sentiment, scores)
        sentiment = adjust_sentiment_based_on_rating(sentiment, rating)


        review["sentiment"] = sentiment
        if sentiment != original_sentiment:
          review["rule_based_override"] = f"{original_sentiment} → {sentiment}"
        else:
          review["rule_based_override"] = None


        # Adjust sentiment_scores based on the number of labels
        num_labels = len(scores)
        if num_labels == 2:
            review["sentiment_scores"] = {
                "негатив": round(scores[0].item(), 3),
                "позитив": round(scores[1].item(), 3),
            }
        else:
            review["sentiment_scores"] = {
                "негатив": round(scores[0].item(), 3),
                "нейтральный": round(scores[1].item(), 3),
                "позитив": round(scores[2].item(), 3),
            }
    else:
        review["sentiment"] = "не определено"
        review["sentiment_scores"] = None

    results.append(review)

# Сохраняем результаты
with open("reviews_with_sentiment.json", "w", encoding="utf-8") as file:
    json.dump(results, file, ensure_ascii=False, indent=4)

print("Анализ тональности завершен! Результаты сохранены в reviews_with_sentiment.json")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Анализ тональности завершен! Результаты сохранены в reviews_with_sentiment.json


**2. Анализ ключевых тем**

In [33]:
!pip install rapidfuzz

In [34]:
from rapidfuzz import fuzz
import json
import re

aspect_keywords = {
    "Качество товара": {
        "positive": [
            "качественный", "надежный", "прочный", "нормально сделан", "сделано хорошо",
            "сделан добротно", "не скрипит", "работает стабильно"
        ],
        "negative": [
            "сломался", "брак", "развалился", "хлипкий", "трещина", "некачественный",
            "разболтался", "шумит", "перестал работать", "бракованный"
        ]
    },
    "Комплектация": {
        "positive": [
            "всё в комплекте", "полный комплект", "насадки есть", "оснащение хорошее",
            "всё как заявлено"
        ],
        "negative": [
            "не положили", "не хватает", "отсутствует", "пустая коробка",
            "чего-то не хватает", "не доложили", "не тот комплект"
        ]
    },
    "Доставка": {
        "positive": [
            "пришло быстро", "пришел вовремя", "доставили раньше", "упакован хорошо",
            "всё целое", "упаковано отлично", "никаких повреждений", "доставка отличная",
            "привезли быстро", "доставка на уровне"
        ],
        "negative": [
            "ждал долго", "задержка", "упаковка повреждена", "мятая коробка",
            "сломали в доставке", "испорчена упаковка"
        ]
    },
    "Соответствие описанию": {
        "positive": [
            "соответствует описанию", "как в описании", "ожидания оправдались",
            "всё как заявлено", "совпадает с фото", "оригинал", "всё так, как описано"
        ],
        "negative": [
            "не соответствует", "не совпадает", "не такое", "вранье", "обман",
            "подделка", "в описании другое", "не как на фото"
        ]
    },
    "Цена": {
        "positive": [
            "дешево", "цена отличная", "доступный", "цена норм", "выгодно",
            "цена приемлемая", "дёшево и сердито", "своих денег стоит",
            "цена хорошая", "дешевле только даром"
        ],
        "negative": [
            "дорого", "переплата", "не стоит этих денег", "цена завышена",
            "расходы большие", "завышенная цена"
        ]
    },
    "Мощность/производительность": {
        "positive": [
            "мощный", "тянет отлично", "производительный", "работает быстро",
            "не греется", "справляется с задачами", "не тормозит"
        ],
        "negative": [
            "не тянет", "слабый", "перегревается", "не справляется",
            "медленный", "греется сильно", "тормозит"
        ]
    },
    "Удобство": {
        "positive": [
            "удобный", "эргономичный", "лёгкий", "компактный", "в руке лежит хорошо",
            "прост в использовании", "интуитивно понятно"
        ],
        "negative": [
            "неудобный", "тяжелый", "непродуманный", "громоздкий", "не ложится в руку",
            "сложно использовать"
        ]
    },
    "Эстетика/внешний вид": {
        "positive": [
            "красивый", "симпатичный", "стильно выглядит", "дизайн приятный",
            "выглядит хорошо", "внешне нравится", "эстетика на уровне"
        ],
        "negative": [
            "уродливый", "дизайн отстой", "выглядит дёшево", "страшный",
            "внешне плохой", "неприятный внешний вид"
        ]
    }
}

# --- Бессмысленные фразы ---
useless_phrases = [
    r"все хорошо", r"доволен", r"понравилось", r"супер", r"отлично",
    r"ок", r"всё нравится", r"все устраивает", r"без нареканий", r"нареканий нет"
]

def clean_phrases(text):
    text = text.lower()
    for phrase in useless_phrases:
        if re.search(phrase, text):
            return ""
    return text.strip()

# --- Сопоставление с аспектами ---
def extract_aspects(text):
    text = text.lower()
    found_pros = []
    found_cons = []

    words = re.findall(r"\w+", text)

    for aspect, polarity_map in aspect_keywords.items():
        for polarity, keywords in polarity_map.items():
            for kw in keywords:
                for word in words:
                    if fuzz.ratio(word, kw) >= 85 or fuzz.partial_ratio(kw, text) > 85:
                        if polarity == "positive":
                            found_pros.append(aspect)
                        elif polarity == "negative":
                            found_cons.append(aspect)
                        break
    return list(set(found_pros)), list(set(found_cons))

# --- Обработка отзывов ---
with open("reviews_with_sentiment.json", "r", encoding="utf-8") as f:
    reviews = json.load(f)

result_rows = []

for review in reviews:
    text = review.get("text", "")
    pros_raw = clean_phrases(review.get("pros", ""))
    cons_raw = clean_phrases(review.get("cons", ""))

    if not pros_raw and not cons_raw and not text.strip():
        continue  # нет полезного текста

    # Раздельный анализ
    pros_text, cons_text = extract_aspects(text)
    pros_pros, cons_pros = extract_aspects(pros_raw)
    pros_cons, cons_cons = extract_aspects(cons_raw)

    final_pros = list(set(pros_text + pros_pros + pros_cons))
    final_cons = list(set(cons_text + cons_pros + cons_cons))


    # Балансировка
    if len(final_pros) >= 2 * len(final_cons):
        final_cons = []
    elif len(final_cons) >= 2 * len(final_pros):
        final_pros = []

    result_rows.append({
        "Оценка": review.get("productValuation"),
        "Достоинства": ", ".join(final_pros) if final_pros else "не выявлены",
        "Недостатки": ", ".join(final_cons) if final_cons else "не обнаружены",
        "Пример отзыва": text
    })

# --- Сохраняем результат ---
with open("analyzed_reviews_output.json", "w", encoding="utf-8") as f_out:
    json.dump(result_rows, f_out, ensure_ascii=False, indent=2)

print("Готово! Проверяй результат в файле analyzed_reviews_output.json")

Готово! Проверяй результат в файле analyzed_reviews_output.json
